# Techniques for Formatting LLM Output

Hello everyone. Getting an LLM to produce factually correct content is only half the battle. For developers, that content is often useless unless it's also in a **perfectly structured format**. If you need a parsable JSON object or a valid SQL query, "close enough" simply doesn't work.

This notebook will demonstrate some common techniques we use to control and format LLM output, progressing from simple requests to a robust, layered, professional workflow.

## The Task and Helper Functions

Our consistent task for this notebook will be to **analyze a user review and extract key information into a JSON object.**

In [ ]:
import litellm
import json
from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 200

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        max_tokens=max_tokens,
        **kwargs
    )

    return response.choices[0].message.content

print("Setup complete. Helper functions and user review are ready.")

In [ ]:
USER_REVIEW = dedent("""
    I am absolutely thrilled with my new 'QuantumLeap' server! It's faster than anything I've ever used.
    The setup was a bit tricky, but the support team was fantastic. I'm giving it a 5 out of 5 stars.
    Highly recommended for any serious data processing work.
""")

## Direct Instruction (The "Please" Method)

This is the most basic approach: simply tell the model what format you want. It's easy but can be unreliable for complex formats.

In [ ]:
prompt_v1 = [
    {
        "role": "user",
        "content": dedent(f"""
        Analyze the following user review and output your findings as JSON.
        Include the product name, a short summary, and a sentiment score.

        <review>
        {USER_REVIEW}
        </review>
        """)
    }
]

response_v1 = get_completion(prompt_v1)

In [ ]:
print(response_v1)

## Few-Shot Prompting (The "Show, Don't Tell" Method)

This is a major step up in reliability. We *show* the model the exact format we want by providing a high-quality example.

In [ ]:
prompt_v2 = [
    {
        "role": "system",
        "content": dedent("""
        You are an expert review analyst.

        Convert user reviews into a structured JSON following the provided examples.
        """)
    },
    {
        "role": "user",
        "content": "The 'StellarSSD' is ok, but the price is a bit too high."
    },
    {
        "role": "assistant",
        "content": json.dumps({
            "product_name": "StellarSSD",
            "summary": "The product is function but overpriced",
            "sentiment_score": 3
        })
    },
    {
        "role": "user",
        "content": USER_REVIEW
    }
]

response_v2 = get_completion(prompt_v2)

In [ ]:
print(response_v2)

## Providing a Template (The "Fill in the Blanks" Method)

This technique gives the model even less room for error. We provide the exact structure and ask the model to fill in the values.

In [ ]:
json_template = {
    "product_name": "",
    "summary": "",
    "sentiment_score": 0
}

prompt_v3 = [
    {
        "role": "system",
        "content": dedent(f"""
        You are an expert review analyst.

        Convert user reviews into a structured JSON following the provided template.

        <template>
        {json.dumps(json_template)}
        </template>
        """)
    },
    {
        "role": "user",
        "content": USER_REVIEW
    }
]

response_v3 = get_completion(prompt_v3)

In [ ]:
print(response_v3)

## API-Level Enforcement

This is the most robust method. We provide a clear schema and use the `response_format` parameter to **guarantee** the output will be a syntactically valid JSON object.

In [ ]:
prompt_v4 = [
    {
        "role": "system",
        "content": dedent("""
        You are an expert review analyst.

        Convert user reviews into a structured JSON following the schema:

        {
            "product_name": str,
            "summary": str,
            "sentiment_score": int(min=1, max=5)
        }

        If the received content is not a review, then respond with the following schema:

        {
            "error": str,
            "original_message": str
        }
        """)
    }
]

response_v4 = get_completion(
    prompt_v4 + [
        {
            "role": "user",
            "content": USER_REVIEW
        }
    ],
    response_format={"type": "json_object"}
)

response_v4_error = get_completion(
    prompt_v4 + [
        {
            "role": "user",
            "content": "I'm feeling great today!"
        }
    ],
    response_format={"type": "json_object"}
)

In [ ]:
print(response_v4)
print(response_v4_error)